In [1]:
import tarfile
with tarfile.open('model.tar.gz', 'w:gz') as f:
    t = tarfile.TarInfo('models')
    t.type = tarfile.DIRTYPE
    f.addfile(t)
    f.add('fastai_model_resnet50.pkl', arcname='fastai_model_resnet50.pkl')

In [4]:
import sagemaker
from sagemaker.utils import name_from_base
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = f'sagemaker/{name_from_base("fastai-model-resnet50")}'
model_artefact = sagemaker_session.upload_data(path='model.tar.gz', bucket=bucket, key_prefix=prefix)
print('Uploaded')

Uploaded


In [45]:
from sagemaker.predictor import Predictor, json_deserializer

class PredictorConfig(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=None, 
                         deserializer=json_deserializer)

In [46]:
import sagemaker
role = sagemaker.get_execution_role()

In [57]:
from sagemaker.pytorch import PyTorch, PyTorchModel

model=PyTorchModel(model_data=model_artefact, name=name_from_base("fastai-model-resnet50"),
    role=role, entry_point='serve.py', framework_version="2.0", py_version="py310", predictor_cls=PredictorConfig, source_dir='src')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-------------!